In [1]:
import torch
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda', index=0)

In [3]:
from SimAttention.model import SimAttention_3
from SimAttention.network.encoder import PCT_Encoder
from SimAttention.network.augmentation import Batch_PointWOLF
from SimAttention.utils.crops import *
from SimAttention.utils.provider import *
from SimAttention.dataloader import ModelNetDataSet
from torch.utils.data import DataLoader

In [4]:
root = '/home/akira/下载/Pointnet2_PyTorch-master/byol_pcl/data/modelnet40_normal_resampled'
dataset = ModelNetDataSet(root)
trainDataLoader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

9843 point clouds found.


In [5]:
train_epoch = 10

In [6]:
aug_method = Batch_PointWOLF()

In [7]:
logger = logging.getLogger(__name__)

In [8]:
online_encoder = PCT_Encoder()
attention_feature_method = get_attention_feature

In [9]:
model = SimAttention_3(aug_method, b_fps, b_get_slice, b_get_cube, b_get_sphere, 
                       online_encoder, attention_feature_method)

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.3)

In [14]:
print('Start training...')
for epoch in range(0, train_epoch):
    print('Epoch {} is running...'.format(epoch))
    model.train()
    for data in trainDataLoader:
        points = data.numpy()
        points = random_point_dropout(points)
        points = random_scale_point_cloud(points)
        points = shift_point_cloud(points)
        points = torch.Tensor(points)
        optimizer.zero_grad()
        loss = model(points)
        print('loss is: ', loss)
        loss.backward(torch.ones_like(loss))
        optimizer.step()
    scheduler.step()

Start training...
Epoch 0 is running.
loss is:  tensor([-2.3842e-07,  2.3842e-07,  3.4571e-06,  7.1526e-07],
       grad_fn=<AddBackward0>)
loss is:  tensor([ 7.1526e-07,  2.9802e-06,  3.5763e-07, -9.5367e-07],
       grad_fn=<AddBackward0>)
loss is:  tensor([3.7789e-05, 1.4305e-06, 3.0994e-06, 8.3447e-07],
       grad_fn=<AddBackward0>)
loss is:  tensor([-4.7684e-07,  1.6689e-06,  1.5497e-06,  2.5034e-06],
       grad_fn=<AddBackward0>)
loss is:  tensor([1.0729e-06, 1.1325e-05, 1.0729e-06, 7.1526e-07],
       grad_fn=<AddBackward0>)
loss is:  tensor([ 5.2452e-06,  2.9802e-06, -4.7684e-07, -1.3113e-06],
       grad_fn=<AddBackward0>)
loss is:  tensor([-9.5367e-07,  2.3842e-07,  7.2718e-06,  2.2650e-06],
       grad_fn=<AddBackward0>)
loss is:  tensor([-2.3842e-07, -2.3842e-07,  1.3828e-05,  2.3842e-07],
       grad_fn=<AddBackward0>)
loss is:  tensor([-4.7684e-07,  2.0266e-06,  2.7418e-06, -3.5763e-07],
       grad_fn=<AddBackward0>)
loss is:  tensor([-1.1921e-07,  1.1921e-07, -2.3842e

KeyboardInterrupt: 